<p style="font-size:30px; font-weight:bold">
Case Study – Comparison of Pan Production with DEALA
</p>

<p style="font-size:12px">
In general, this case study demonstrates how the DEALA package can be used and how the DEALA method should be applied to calculate the economic impacts of production systems.
</p>

<p style="font-size:12px">
As an example, three different pans are compared regarding their economic and environmental impacts. The following pans are considered:
</p>

<p style="font-size:12px">
1. A conventional non-stick pan using PTFE as a coating<br>
2. A re-coatable non-stick pan using PTFE as a coating<br>
3. A stainless steel pan
</p>

<p style="font-size:12px">
The notebook is divided into five steps, which are:
</p>

1. [Initialization of the project](#1-initalization-of-the-project)  
2. [Import of the background system](#2-import-background-system)  
3. [Import of the intermediate system](#3-import-intermediate-system)  
4. [Import of the foreground system](#4-import-foreground-system)  



# 1. Initalization of the project

<p style="font-size:12px">
The initialization step includes two substeps:
</p>

<ol style="font-size:12px; padding-left: 20px">
  <li>The required Python modules and packages must be imported and defined.</li>
  <li>The project is set up, including the definition of file paths to external data and the assumptions made for the case study.</li>
</ol>


## 1.1 load python packages

<p style="font-size:18px">
To load all required Python modules and packages, you first need to install the <code>DEALA</code> package via pip. Use <code>pip install deala</code> for this.
</p>

In [ ]:
# initializatiin of packages
import brightway2 as bw
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bw2io import *
from tqdm import *
import math
import toml
import sys
from pathlib import Path
# change the path here to wherever you stored the Regioinvent Python package 
# only needed if you've not installed through pip
import regioinvent
from bw2data import *
from bw2data.parameters import (
    ActivityParameter,
    DatabaseParameter,
    ProjectParameter,
    Group,
)
import deala as de
from deala import deala_io
deala_io_instance = deala_io()


## 1.2 Set up project

<p style="font-size:12px">
The following aspects are defined in this substep:
</p>

<ol style="font-size:12px; padding-left: 20px">
  <li><code>set_current</code>: Defines the name of the project and loads all associated data sets. In this assessment, the project is named <code>Pan_Production</code>.</li><br><br>
  
  <li><code>FP_ecoinvent</code>: Specifies the file path to the folder containing the ecoinvent database required to run the case study.</li><br><br>
  
  <li><code>directory_deala</code>: Specifies the path to the DEALA package to access all required files.</li><br><br>
  
  <li><code>current_directory</code>: Specifies the directory where this notebook is stored.</li><br><br>
  
  <li><code>repository_main_path</code>: Specifies the path to the main repository.</li><br><br>
  
  <li><code>FP_mapping</code>: Specifies the file path to the TOML file used for mapping life cycle impact assessment methods, energy, and transport data sets to predefined settings.</li><br><br>
  
  <li><code>FP_transport_matrices</code>, <code>FP_transport_matrices_land</code>, <code>FP_transport_matrices_sea</code>, <code>FP_transport_matrices_harbor</code>: Specifies the file paths to the transport matrices required for the calculations. With DEALA, transport distances between capital cities of various countries can be considered.</li><br><br>
  
  <li><code>rf_pan</code>: Defines the reference flow(s) for the case study. In this assessment, the reference flow is one pan for each of the three pan types.</li><br><br>
  
  <li><code>dict_countries</code>: Defines the countries to be considered in the intermediate and foreground systems as a dictionary. This case study includes Germany, China, and the USA, in addition to global activities (used as default).</li><br><br>
  
  <li><code>dict_scenarios</code>: Defines the scenarios to be considered in the case study, including time period and scenario name, as a dictionary. Two scenarios are used in this case: the Status Quo (2024) and a future scenario (2030) based on the REMIND-SSP2-base scenario. The key nomenclature is selected for automatic regionalization and linking.</li><br><br>
  
  <li><code>percentage_rate_buildings</code>: Defines the percentage rate for building investments to be considered in the economic assessment (depreciation). In this assessment, a rate of 3% is used.</li><br><br>
  
  <li><code>percentage_rate_machinery</code>: Defines the percentage rate for machinery investments to be considered in the economic assessment (depreciation). In this assessment, a rate of 5% is used.</li><br><br>
  
  <li><code>percentage_rate_insurance</code>: Defines the percentage rate for insurance costs based on investments in machinery and buildings. In this assessment, a rate of 1% is used.</li><br><br>
  
  <li><code>dict_interest</code>: Defines the national interest rates per country as a dictionary. These rates are used in the economic assessment for investments in machinery and buildings.</li><br><br>
  
  <li><code>dict_taxes</code>: Defines the profit tax rates per country. The tax rates are based on data from <a href="https://taxsummaries.pwc.com/" target="_blank">PwC</a>.</li><br><br>
  
  <li><code>percentage_rate_administration</code>: Defines the percentage rate for administration costs based on personnel costs. In this assessment, a rate of 25% is used.</li><br><br>
  
  <li><code>percentage_rate_research_development</code>: Defines the percentage rate for research and development based on total costs. In this assessment, a rate of 2% is used.</li><br><br>
  
  <li><code>percentage_rate_warranty</code>: Defines the percentage rate for warranty costs based on total costs. In this assessment, a rate of 3% is used.</li><br><br>
  
  <li><code>percentage_margin</code>: Defines the profit margin of the pan producer used to calculate the final product price for each country.</li><br><br>
  
  <li><code>list_end_products</code>: Defines a list of end products (e.g., stainless steel pan and non-stick pan) required for the calculation.</li><br><br>
  
  <li><code>nr_work_days</code>: Defines the number of working days per year that the facility operates. In this assessment, the facility operates 320 days per year.</li><br><br>
  
  <li><code>nr_work_hours</code>: Defines the number of working hours per day that the facility operates. In this assessment, the facility operates 16 hours per day.</li><br><br>
  
  <li><code>nr_days_per_worker</code>: Defines the number of working days per year per worker. In this assessment, a worker operates 210 days per year.</li><br><br>
  
  <li><code>length_of_shift</code>: Defines the length of a shift in hours. In this assessment, a shift lasts 8 hours.</li><br><br>
  
  <li><code>base_year</code>: Defines the base year for the economic assessment, meaning all economic values are normalized to this year. In this assessment, the base year is 2023.</li><br><br>
  
  <li><code>FP_premise</code>: Specifies the file path to the file containing the key for Premise, which is required to manipulate the ecoinvent database.</li>
</ol>


In [ ]:
# project name
PROJECT_NAME = 'pan_production_regiopremise1'

# Ecoinvent datasets folder
ECOINVENT_PATH = Path(os.path.expanduser("~")) / 'Desktop' / 'ecoinvent 3.9.1_cutoff_ecoSpold02' / 'datasets'
ECOINVENT_VERSION = '3.9.1'

NAME_ECOINVENT_DB = f'ecoinvent_{ECOINVENT_VERSION}-cutoff_remind_SSP2-Base_ecoSpold02'
NAME_ECOINVENT_DB_SHORT = f'ecoinvent_{ECOINVENT_VERSION}-cutoff_'

PREMISE_KEY_PATH = Path(os.path.expanduser("~")) / 'Desktop' / "key_premise.toml"

# Filepath to trade database
TRADE_DATABASE_PATH = Path(os.path.expanduser("~")) / 'Downloads' / 'trade_data.db'

In [ ]:
#Create/load project
# project_name ='regiopremise_test_small2'
project_name = PROJECT_NAME
bw.projects.set_current(project_name)

# mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
# FP_ecoinvent = os.path.join(os.path.expanduser("~"), "Desktop", "ecoinvent 3.9.1_cutoff_ecoSpold02", "datasets")
FP_ecoinvent = ECOINVENT_PATH

# Definition of ecoinvent version
# ei_version = '3.9.1'
ei_version = ECOINVENT_VERSION

# Definition of ecoinvent version for first year
# name_ecoinvent= 'ecoinvent_3.9.1-cutoff_remind_SSP2-Base_ecoSpold02'
name_ecoinvent= NAME_ECOINVENT_DB

# Definition of ecoinvent name short
# name_ecoinvent_short= 'ecoinvent_3.9.1-cutoff_'
name_ecoinvent_short= NAME_ECOINVENT_DB_SHORT

# Get the directory of the deala package
directory_deala = os.path.dirname(de.__file__)

# Get the current working directory
current_directory = os.getcwd()

# Determine the main path of the repository
repository_main_path = os.path.abspath(os.path.join(current_directory, os.pardir))

# filepath to mapping file
FP_mapping = os.path.join(directory_deala, 'files', "linking", "mappings.toml")

# filepath to transport matrices
FP_transport_matrices = os.path.join(repository_main_path, 'files', "linking", "distance_matrix.xlsx")

# filepath to transport matrices
FP_transport_matrices_land = os.path.join(repository_main_path, 'files', "linking", "distance_matrix_land.xlsx")

# filepath to transport matrices
FP_transport_matrices_sea = os.path.join(repository_main_path, 'files', "linking", "distance_matrix_sea.xlsx")

# filepath to transport matrices
FP_transport_matrices_land_harbor = os.path.join(repository_main_path, 'files', "linking", "distance_matrix_land_harbor.xlsx")

# filepath to PPP data
FP_material_regionalized= os.path.join(repository_main_path, 'files', "linking", "PPP_countries.xlsx")

# Definition of reference flows
rf_pan= 1

# definition of countries producing the pans
dict_countries={

    'China': 'CN',
    'France': 'FR',
    'Germany': 'DE',
    'India': 'IN',
    'Italy': 'IT',
    'United States of America': 'US',
    'Turkey': 'TR',
    'Netherlands': 'NL',
    'Poland': 'PL',
    'Belgium': 'BE',
    'Czechia': 'CZ',
}

# Definition of the scenarios required for the case study in a dict
dict_scenarios={
    'remind_SSP2-Base_ecoSpold02':2024,
    'remind_SSP2-PkBudg1150_2030':2030,
    'remind_SSP2-Base_2030':2030
}

#definition of the percentage rate for maintenance and repair 
percentage_rate_buildings= 'maintenance and repair - 3 percent'
percentage_rate_machinery= 'maintenance and repair - 5 percent'

#definition of the percentage rate for insurance
percentage_rate_insurance= 'insurance - 1 percent'

# #definition of the percentage rate for interest
dict_interest={
    'GLO':'interest (internal) - 5 percent',  
    'CN':'interest (internal) - 3.5 percent',       
    'DE':'interest (internal) - 4 percent',            
    'FR':'interest (internal) - 4 percent',     
    'IN':'interest (internal) - 6.5 percent',
    'IT':'interest (internal) - 4 percent',                 
    'US':'interest (internal) - 5.25 percent',
    'TR':'interest (internal) - 20.75 percent',
    'NL':'interest (internal) - 4 percent',
    'PL':'interest (internal) - 6.5 percent',
    'BE':'interest (internal) - 4 percent',
    'CZ':'interest (internal) - 7 percent',
}

# definition of the percentage rate for taxes
dict_taxes={
    'GLO':'taxes - 20.6 percent',
    'CN':'taxes - 15 percent',          #https://taxsummaries.pwc.com/china/corporate/taxes-on-corporate-income
    'DE':'taxes - 15.8 percent',        #https://taxsummaries.pwc.com/germany/corporate/taxes-on-corporate-income
    'FR':'taxes - 25 percent',          #https://taxsummaries.pwc.com/france/corporate/taxes-on-corporate-income
    'IN':'taxes - 30 percent',          #https://taxsummaries.pwc.com/india/corporate/taxes-on-corporate-income
    'IT':'taxes - 24 percent',          #https://taxsummaries.pwc.com/italy/corporate/taxes-on-corporate-income
    'US':'taxes - 21 percent',          #https://taxsummaries.pwc.com/united-states/corporate/taxes-on-corporate-income
    'TR':'taxes - 25 percent',          #https://taxsummaries.pwc.com/turkey/corporate/taxes-on-corporate-income
    'NL':'taxes - 25.8 percent',        #https://taxsummaries.pwc.com/netherlands/corporate/taxes-on-corporate-income
    'PL':'taxes - 19 percent',          #https://taxsummaries.pwc.com/poland/corporate/taxes-on-corporate-income
    'BE':'taxes - 25 percent',          #https://taxsummaries.pwc.com/belgium/corporate/taxes-on-corporate-income
    'CZ':'taxes - 19 percent',          #https://taxsummaries.pwc.com/czech-republic/corporate/taxes-on-corporate-income
}

#definition of the percentage rate for administration
percentage_rate_administration= 'administration - 25 percent'

#definition of the percentage rate for research and development
percentage_rate_research_development= 'research and development - 2 percent'

#definition of the percentage rate for warranty
percentage_rate_warranty= 'warranty - 3 percent'

#definition of the percentage rate for margin on pans
percentage_margin= 0.5      # 50 percent margin on the production costs

#definition of the end products
list_end_products = ['Non-stick pan', 'Stainless steel pan']

# Definition of the working days per year
nr_work_days = 320

# Definition of the working hours per day
nr_work_hours = 16

# Definition of number of days per worker
nr_days_per_worker=210

# Definition of the length of shift in hours
length_of_shift=8

# Definition of the base year for normalization of costs
base_year = 2023

# Definition of the key for the premise database
# desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "key_premise.toml")
desktop_path = PREMISE_KEY_PATH
config = toml.load(desktop_path)
key_premise=config['auth']['key_premise']

# Definition of filepath to trade database
# FP_trade_database = os.path.join(os.path.expanduser("~"), "Downloads", "trade_data.db")
FP_trade_database = TRADE_DATABASE_PATH

# Definition of cut-off for regioinvent
cutoff_regioinvent= 0.8 # Zum TESTEN sonst 0.8

# Definition of price calculation method
price_calculation_method= "real"

# Definition of method for calculation of real prices
method_calculation_real_prices= "reg"

# Definition of filepath to Energy database
FP_energy_database = os.path.join(directory_deala, "files", "Excel_datasets", "DB_Energy.xlsx")

# Definition of filepath to input materials
FP_input_materials = os.path.join(directory_deala, "files", "Excel_datasets", "pan_production", "input_materials_regioinvent.xlsx")

# Definition of filepath to output materials
FP_output_materials = os.path.join(directory_deala, "files", "Excel_datasets", "pan_production", "output_materials.xlsx")

# Definition of filepath to non-stick pan (conventional) production dataset
FP_pan_non_stick_conventional = os.path.join(directory_deala, "files", "Excel_datasets", "pan_production", "Excel_BW2_non_stick_conventional.xlsx")

# Definition of filepath to non-stick pan (recoatable) production dataset
FP_pan_non_stick_recoatable = os.path.join(directory_deala, "files", "Excel_datasets", "pan_production", "Excel_BW2_non_stick_pan_recoatable.xlsx")

# Definition of filepath to stainless steel pan production dataset
FP_pan_stainless_steel = os.path.join(directory_deala, "files", "Excel_datasets", "pan_production", "Excel_BW2_stainless_steel.xlsx")

# Definition of name of last process
last_process_name = 'Attaching handle'



# 2. Import background system

<p style="font-size:12px">
The background system consists of two external databases that must be imported into the initialized project. These databases are:
</p>

<ul style="font-size:12px; padding-left: 20px">
  <li>Ecoinvent database</li>
  <li>DEALA database</li>
</ul>

<p style="font-size:12px">
The last two databases are created for the defined scenarios to reflect temporal differences. To manipulate the ecoinvent database, the <code>premise</code> package is used.
</p>

<p style="font-size:12px">
Additionally, life cycle impact assessment (LCIA) methods for both environmental and economic assessments are imported, along with the required databases for elementary flows (<code>biosphere</code> and <code>marketsphere</code>).
</p>


## 2.1 Import biosphere database and environmental impact categories

In [ ]:
# Install biosphere database and impact assessment methods
bw.bw2setup()

## 2.2 Import ecoinvent

In [ ]:
# Install ecoinvent database
if name_ecoinvent in bw.databases:
    print("Database has already been imported.")
else:
    fpei39cut = FP_ecoinvent
    ei39cut = bw.SingleOutputEcospold2Importer(
        fpei39cut, name_ecoinvent
    )
    ei39cut
    ei39cut.apply_strategies()
    ei39cut.statistics()
    ei39cut.write_database()  # writes database to Brightway2 project

## 2.3 Import Premise databases

In [ ]:
deala_io.create_premise_databases(dict_scenarios, key_premise, "ecospold", FP_ecoinvent, ECOINVENT_VERSION, overwrite=False)

# 2.3 Regionalization of ecoinvent databases using regiopremise

The Python package regiopremise is used to regionalize the different ecoinvent databases (as defined in dict_scenario). 

In [ ]:
for scenario in dict_scenarios:
    regio = regioinvent.Regiopremise(
        bw_project_name=project_name,
        premise_database_name=name_ecoinvent_short+scenario,
        ecoinvent_database_name=name_ecoinvent,
        ecoinvent_version=ei_version,
    )

    regio.fix_iam_location_codes()
    regio.spatialize_my_premise()
    regio.import_fully_regionalized_impact_method(lcia_method='all')
    regio.regionalize_premise_with_trade(
        trade_database_path=FP_trade_database,
        regioinvent_database_name='regioinvent_'+scenario,
        cutoff=cutoff_regioinvent,
    )

## 2.4 Import DEALA

To import the marketsphere and the DEALA databases, three steps must be performed:
</p>

<ol style="font-size:12px; padding-left: 20px">
  <li>
    <code>deala_setup</code>: This function loads the marketsphere database, including all required marketsphere flows to calculate economic impacts, as well as life cycle impact assessment methods. In this assessment, three different characterization models from DEALA can be selected for the calculations. This step is comparable to <code>bw.setup()</code>.
  </li><br><br>
  
  <li>
    <code>import_deala_activities</code>: In this step, predefined datasets representing prices for personnel, construction, materials, etc. are used to create the corresponding DEALA activities in the database. This includes consideration of the <code>base_year</code> and the different <code>scenarios</code> to generate scenario-dependent databases. The respective data sheets are stored in the DEALA directory. Additionally, it must be specified whether to use nominal or real prices, and how future prices should be approximated—either by regression analysis or geometric average.
  </li><br><br>
  
  <li>
    <code>create_default_DEALA_activities</code>: In this final step, default DEALA activities are added to the databases. These are activities for which no specific price or economic value is known in advance, as they depend on factors such as investments, total costs, or CO₂ emissions.
  </li>
</ol>

In [ ]:
# Install the DEALA impact assessment method and marketsphere database
deala_io_instance.deala_setup(overwrite=True)

In [ ]:
#create deala activities that are linked to cost data
deala_io_instance.import_DEALA_activities_fast(base_year, dict_scenarios, directory_deala, price_calculation=price_calculation_method, method_calc_r=method_calculation_real_prices, modus=None)
deala_io_instance.import_PPP_DEALA_activities(FP_material_regionalized)

In [ ]:
#create remaining DEALA activities

deala_io_instance.create_default_DEALA_activities_fast('DEALA_activities_', dict_scenarios, overwrite=False)

# 3. Import intermediate system

<p style="font-size:12px">
The intermediate system consists of four databases that must be imported into the initialized project. These databases are:
</p>

<ul style="font-size:12px; padding-left: 20px">
  <li>Energy database</li>
  <li>Transport database</li>
  <li>Input materials database</li>
  <li>Output materials database</li>
</ul>

<p style="font-size:12px">
These databases contain activities that are combinations of exchanges from the DEALA and ecoinvent databases.
</p>

<p style="font-size:12px">
The energy database includes datasets for electricity and heat from gas, differentiated by household and non-household consumers, various countries, and different consumption levels.
</p>

<p style="font-size:12px">
The transport database includes datasets representing distances and transport between capital cities. Two transport modes are considered: container ship and lorry.
</p>

<p style="font-size:12px">
The input materials database includes all materials required for pan production.
</p>

<p style="font-size:12px">
The output materials database includes all co-products and waste streams that must be further processed.
</p>


## 3.1 Import energy databases

In [ ]:
# allocation of datasets in Energy database to types in DEALA Activities to create regionalized activities
dict_energy=toml.load(FP_mapping)["energy"]

# Delete activities from energy-related ecoinvent databases to reduce vacuuming time

list_databases = []

# Step 1: Retrieve a list of all existing databases in Brightway2
for database in bw.databases:
    list_databases.append(database)

# Step 2: Delete activities from databases whose names contain both 'ecoinvent' and 'Energy'
for database in list_databases:
    if 'ecoinvent' in database and 'Energy' in database:
        for activity in bw.Database(database):
            activity.delete()  # Remove each activity from the database
        print(database)  # Print the name of the database after deletion
#         del bw.databases[database]  # Optionally, delete the database entry from Brightway2

# Step 3: Load default energy datasets for the defined scenarios from Excel
dict_activities = deala_io_instance.create_target_databases(
    FP_energy_database,
    'Energy'
)

# Step 4: Identify dependent activities between the Energy databases and DEALA databases
dict_databases, dict_matches, lst_DB_DEALA, lst_DB_Energy = deala_io.identify_dependent_activities('Energy')

# Step 5: Identify matches between activities in the target energy database and ecoinvent
dict_matches = deala_io.identify_matches(lst_DB_Energy[0])

# Step 6: Copy default activities to regionalized activities and add respective DEALA activities to Energy activities
deala_io_instance.copy_and_add_DEALA_activity_fast(
    dict_databases=dict_databases,
    dict_target=dict_energy,
    dict_activities=dict_activities,
    overwrite=False,
    debug=True
)

# # Step 7: Update ecoinvent exchanges in the energy databases so that they match the defined location
ecoinvent_dbs = [
    db for db in bw.databases
    if "ecoinvent" in db and "Energy" not in db and "Transport" not in db
]

stats = deala_io_instance.update_exchanges_replace_consistent_fast(
    dict_matches=dict_matches,
    lst_DB_energy=lst_DB_Energy,
    lst_DB_ecoinvent=ecoinvent_dbs,
    debug=False
)
print(stats)

## 3.2 Import transport databases

In [ ]:
#this function creates the transport databases for the defined scenarios
deala_io_instance.build_transport_databases_fast(
    dict_scenarios=dict_scenarios,
    FP_transport_matrices_land=FP_transport_matrices_land,
    FP_transport_matrices_sea=FP_transport_matrices_sea,
    FP_transport_matrices_land_harbor=FP_transport_matrices_land_harbor,
    name_ecoinvent_prefix=name_ecoinvent_short,
    overwrite=True,
)

## 3.3 import input materials

In [ ]:
#import input materials database
databases=[db for db in bw.databases if 'Energy']
for scenario in dict_scenarios:
    for db in databases:
        if 'Energy' in db and 'regionalized' in db and scenario in db:
            name_add=" regionalized"
        elif 'Energy' in db and 'regionalized' not in db and scenario in db:
            name_add=""
        else:
            continue
        imp = bw.ExcelImporter(FP_input_materials)
        imp.apply_strategies()
        imp.match_database(fields=('name', 'unit', 'location', 'reference product'))
        imp.match_database(NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('regioinvent_' + scenario, fields=('name','unit','location', 'reference product'))
        imp.match_database('DEALA_activities_' + scenario, fields=('name','location', 'reference product'))
        imp.match_database('Energy_' + NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('biosphere3', fields=('name','unit','categories'))
        imp.statistics()
        imp.write_database()
        db = bw.Database(imp.db_name).rename(imp.db_name + '_' + scenario + name_add)
        # regionalization of activities in the databases
        deala_io_instance.regionalize_process_input_fast(db.name, dict_countries, commit_every=5000)

## 3.4 import output materials

In [ ]:
#import output materials database
databases=[db for db in bw.databases if 'Energy']
for scenario in dict_scenarios:
    for db in databases:
        if 'Energy' in db and 'regionalized' in db and scenario in db:
            name_add=" regionalized"
        elif 'Energy' in db and 'regionalized' not in db and scenario in db:
            name_add=""
        else:
            continue
        target_db_name = f"output_materials_{scenario }{name_add}"
        if target_db_name in bw.databases:
            for activity in bw.Database(target_db_name):
                activity.delete()

        imp = bw.ExcelImporter(FP_output_materials)
        imp.apply_strategies()
        imp.match_database(fields=('name', 'unit', 'location', 'reference product'))
        imp.match_database(NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('DEALA_activities_' + scenario, fields=('name','location', 'reference product'))
        imp.match_database('Energy_' + NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('biosphere3', fields=('name','unit','categories'))
        imp.statistics()
        imp.write_database()
        db = bw.Database(imp.db_name).rename(imp.db_name + '_' + scenario + name_add)
        # regionalization of activities in the databases
        deala_io_instance.regionalize_process_outputmaterials_fast(db.name)

# 4. Import foreground system

<p style="font-size:12px">
The foreground system consists of three databases representing the three different pans that must be imported into the initialized project. These databases are:
</p>

<ul style="font-size:12px; padding-left: 20px">
  <li>Non-stick pan (conventional)</li>
  <li>Non-stick pan (recoatable)</li>
  <li>Stainless steel pan</li>
</ul>

<p style="font-size:12px">
The databases are loaded from pre-defined Excel sheets, including global representations of the production steps. These databases are created for the defined scenarios and subsequently regionalized. After regionalization, the corresponding economic DEALA activities are added to the activities in the databases. For this step, the following priority is applied:
</p>

<ol style="font-size:12px; padding-left: 20px">
  <li>Add DEALA activities representing maintenance and repair for buildings and machines (investment-dependent).</li>
  <li>Add DEALA activities representing insurance (investment-dependent).</li>
  <li>Add DEALA activities representing interest (investment-dependent). The investment is divided by 0.5, as the average interest method is applied in this case study.</li>
  <li>Add DEALA activities representing administration (personnel cost-dependent).</li>
  <li>Add DEALA activities representing research and development (total cost-dependent, excluding taxes and warranty).</li>
  <li>Add DEALA activities representing warranty (total cost-dependent, excluding taxes and revenues from co-products).</li>
  <li>Add DEALA activities representing the prices of end products to the activity producing the end product (total cost-dependent).</li>
  <li>Add DEALA activities representing taxes on profit (total profit-dependent).</li>
</ol>



## 4.1 Non-stick pan (conventional)

In [ ]:
### import pan production processes ###
databases=[db for db in bw.databases if 'Energy']
for scenario in dict_scenarios:
    for db in databases:
        if 'Energy' in db and 'regionalized' in db and scenario in db:
            name_add=" regionalized"
        elif 'Energy' in db and 'regionalized' not in db and scenario in db:
            name_add=""
        else:
            continue
        imp = bw.ExcelImporter(FP_pan_non_stick_conventional)
        imp.apply_strategies()
        imp.match_database(fields=('name', 'unit', 'location', 'reference product'))
        imp.match_database(NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('DEALA_activities_' + scenario, fields=('name','location', 'reference product'))
        imp.match_database('input_materials_' + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('output_materials_' + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('Energy_' + NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('biosphere3', fields=('name','unit','categories'))
        imp.statistics()
        imp.write_database()
        db = bw.Database(imp.db_name).rename(imp.db_name + '_' + scenario + name_add)


        ### regionalization of activities in the databases ###
        deala_io_instance.regionalize_processes_foreground(db.name, dict_countries)


        ### add the investment-dependent deala activities to the processes of the foreground system ###
        db_deala=bw.Database('DEALA_activities_' + scenario)
        dict_machine, dict_buildings, total_investments = deala_io_instance.calculate_investments_buildings_machine(db, 'depreciation (linear) - machinery and equipment', 'depreciation (linear) - Advanced manufacturing facility') # calculate the investments for buildings and machinery
        # add DEALA activities representing maintenance and repair for buildings
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_buildings, percentage_rate_buildings)
        # add DEALA activities representing maintenance and repair for machines and equipment
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_machine, percentage_rate_machinery)
        # add DEALA activities representing insurance dependent on all investments
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_investments, percentage_rate_insurance)
        # add DEALA activities representing interest dependent on all investments
        for location in dict_interest:
            deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_investments, dict_interest[location], amount=0.5, location=location)

        
        ### add the deala activities depending on personnel cost to the processes of the foreground system ###
        # calculate the investments for buildings and machines
        dict_personnel = deala_io_instance.calculate_personnel_cost_processes(db)
        # add DEALA activities representing administration
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_personnel, percentage_rate_administration, keyword='personnel cost')

        
        ### add the deala activities depending on total cost without taxes and warranty to the processes of the foreground system ###
        # calculate the total cost
        total_cost = deala_io_instance.calculate_total_cost_processes(db)
        # # add DEALA activities representing research and development
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_cost, percentage_rate_research_development, keyword='total cost')


        ### add the deala activities depending on total cost without taxes to the processes of the foreground system ###
        # calculate the total cost of processes
        total_cost = deala_io_instance.calculate_total_cost_processes_wo_co_products(db)
        # add DEALA activities representing warranty
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_cost, percentage_rate_warranty, keyword='total cost')


        # # add deala activity representing price of end product
        # deala_io_instance.calculate_price_based_on_margin(db, db_deala, list_end_products, percentage_margin=percentage_margin, overwrite=True)


        # ### add the deala activities representing taxes on profit to the processes of the foreground system leading to products which are selled on the market
        # # calculate the profit 
        # total_profit=deala_io_instance.calculate_profit_processes(db, last_process_name)
        # # add DEALA activities representing taxes
        # for location in dict_taxes:
        #     # calculate the dependent DEALA activity for taxes
        #     if total_profit[(last_process_name, location)]<0:
        #         deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_profit, dict_taxes[location], location=location, keyword='profit', amount=-1)



## 4.2 Non-stick pan (recoatable)

In [ ]:
### import pan production processes ###
databases=[db for db in bw.databases if 'Energy']
for scenario in dict_scenarios:
    for db in databases:
        if 'Energy' in db and 'regionalized' in db and scenario in db:
            name_add=" regionalized"
        elif 'Energy' in db and 'regionalized' not in db and scenario in db:
            name_add=""
        else:
            continue
        imp = bw.ExcelImporter(FP_pan_non_stick_recoatable)
        imp.apply_strategies()
        imp.match_database(fields=('name', 'unit', 'location', 'reference product'))
        imp.match_database(NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('DEALA_activities_' + scenario, fields=('name','location', 'reference product'))
        imp.match_database('input_materials_' + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('output_materials_' + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('Energy_' + NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('biosphere3', fields=('name','unit','categories'))
        imp.statistics()
        imp.write_database()
        db = bw.Database(imp.db_name).rename(imp.db_name + '_' + scenario + name_add)


        ### regionalization of activities in the databases ###
        deala_io_instance.regionalize_processes_foreground(db.name, dict_countries)


        ### add the investment-dependent deala activities to the processes of the foreground system ###
        db_deala=bw.Database('DEALA_activities_' + scenario)
        dict_machine, dict_buildings, total_investments = deala_io_instance.calculate_investments_buildings_machine(db, 'depreciation (linear) - machinery and equipment', 'depreciation (linear) - Advanced manufacturing facility') # calculate the investments for buildings and machinery
        # add DEALA activities representing maintenance and repair for buildings
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_buildings, percentage_rate_buildings)
        # add DEALA activities representing maintenance and repair for machines and equipment
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_machine, percentage_rate_machinery)
        # add DEALA activities representing insurance dependent on all investments
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_investments, percentage_rate_insurance)
        # add DEALA activities representing interest dependent on all investments
        for location in dict_interest:
            deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_investments, dict_interest[location], amount=0.5, location=location)

        
        ### add the deala activities depending on personnel cost to the processes of the foreground system ###
        # calculate the investments for buildings and machines
        dict_personnel = deala_io_instance.calculate_personnel_cost_processes(db)
        # add DEALA activities representing administration
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_personnel, percentage_rate_administration, keyword='personnel cost')

        
        ### add the deala activities depending on total cost without taxes and warranty to the processes of the foreground system ###
        # calculate the total cost
        total_cost = deala_io_instance.calculate_total_cost_processes(db)
        # add DEALA activities representing research and development
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_cost, percentage_rate_research_development, keyword='total cost')


        ### add the deala activities depending on total cost without taxes to the processes of the foreground system ###
        # calculate the total cost of processes
        total_cost = deala_io_instance.calculate_total_cost_processes_wo_co_products(db)
        # add DEALA activities representing warranty
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_cost, percentage_rate_warranty, keyword='total cost')


        # # add deala activity representing price of end product
        # deala_io_instance.calculate_price_based_on_margin(db, db_deala, list_end_products, percentage_margin=percentage_margin, overwrite=True)


        # ### add the deala activities representing taxes on profit to the processes of the foreground system leading to products which are selled on the market
        # # calculate the profit 
        # total_profit=deala_io_instance.calculate_profit_processes(db, last_process_name)
        # # add DEALA activities representing taxes
        # for location in dict_taxes:
        #     # calculate the dependent DEALA activity for taxes
        #     if total_profit[(last_process_name, location)]<0:
        #         deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_profit, dict_taxes[location], location=location, keyword='profit', amount=-1)

## 4.3 Stainless steel pan

In [ ]:
### import pan production processes ###
databases=[db for db in bw.databases if 'Energy']
for scenario in dict_scenarios:
    for db in databases:
        if 'Energy' in db and 'regionalized' in db and scenario in db:
            name_add=" regionalized"
        elif 'Energy' in db and 'regionalized' not in db and scenario in db:
            name_add=""
        else:
            continue
        imp = bw.ExcelImporter(FP_pan_stainless_steel)
        imp.apply_strategies()
        imp.match_database(fields=('name', 'unit', 'location', 'reference product'))
        imp.match_database(NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('DEALA_activities_' + scenario, fields=('name','location', 'reference product'))
        imp.match_database('input_materials_' + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('output_materials_' + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('Energy_' + NAME_ECOINVENT_DB_SHORT + scenario + name_add, fields=('name','unit','location', 'reference product'))
        imp.match_database('biosphere3', fields=('name','unit','categories'))
        imp.statistics()
        imp.write_database()
        db = bw.Database(imp.db_name).rename(imp.db_name + '_' + scenario + name_add)


        ### regionalization of activities in the databases ###
        deala_io_instance.regionalize_processes_foreground(db.name, dict_countries)


        ### add the investment-dependent deala activities to the processes of the foreground system ###
        db_deala=bw.Database('DEALA_activities_' + scenario)
        dict_machine, dict_buildings, total_investments = deala_io_instance.calculate_investments_buildings_machine(db, 'depreciation (linear) - machinery and equipment', 'depreciation (linear) - Advanced manufacturing facility') # calculate the investments for buildings and machinery
        # add DEALA activities representing maintenance and repair for buildings
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_buildings, percentage_rate_buildings)
        # add DEALA activities representing maintenance and repair for machines and equipment
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_machine, percentage_rate_machinery)
        # add DEALA activities representing insurance dependent on all investments
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_investments, percentage_rate_insurance)
        # add DEALA activities representing interest dependent on all investments
        for location in dict_interest:
            deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_investments, dict_interest[location], amount=0.5, location=location)

        
        ### add the deala activities depending on personnel cost to the processes of the foreground system ###
        # calculate the investments for buildings and machines
        dict_personnel = deala_io_instance.calculate_personnel_cost_processes(db)
        # add DEALA activities representing administration
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, dict_personnel, percentage_rate_administration, keyword='personnel cost')

        
        ### add the deala activities depending on total cost without taxes and warranty to the processes of the foreground system ###
        # calculate the total cost
        total_cost = deala_io_instance.calculate_total_cost_processes(db)
        # add DEALA activities representing research and development
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_cost, percentage_rate_research_development, keyword='total cost')


        ### add the deala activities depending on total cost without taxes to the processes of the foreground system ###
        # calculate the total cost of processes
        total_cost = deala_io_instance.calculate_total_cost_processes_wo_co_products(db)
        # add DEALA activities representing warranty
        deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_cost, percentage_rate_warranty, keyword='total cost')


        # # add deala activity representing price of end product
        # deala_io_instance.calculate_price_based_on_margin(db, db_deala, list_end_products, percentage_margin=percentage_margin, overwrite=True)


        # ### add the deala activities representing taxes on profit to the processes of the foreground system leading to products which are selled on the market
        # # calculate the profit 
        # total_profit=deala_io_instance.calculate_profit_processes(db, last_process_name)
        # # add DEALA activities representing taxes
        # for location in dict_taxes:
        #     # calculate the dependent DEALA activity for taxes
        #     if total_profit[(last_process_name, location)]<0:
        #         deala_io_instance.dependent_DEALA_activity_percentage_rate(db, db_deala, total_profit, dict_taxes[location], location=location, keyword='profit', amount=-1)